<a href="https://www.kaggle.com/code/aabdollahii/fine-tuning-a-gpt-2-model-for-a-custom-chatbot?scriptVersionId=265223498" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv


<div style="background-color: #013220; color: #EAEAEA; border-radius: 15px; padding: 30px; font-family: 'Helvetica Neue', sans-serif; border: 1px solid #388E3C;">

<div style="text-align: center; font-size: 32px; font-weight: bold; color: #A5D6A7; padding-bottom: 15px; border-bottom: 2px solid #388E3C; margin-bottom: 25px;">
        Fine-Tuning a GPT-2 Model for a Custom Chatbot
    </div>

<div style="font-size: 20px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Project Overview
    </div>
    <div style="font-size: 16px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        Welcome to this hands-on guide to fine-tuning a Large Language Model (LLM)! In this project, we will transform a general-purpose language model, the powerful GPT-2, into a specialized chatbot. Our goal is to leverage a dataset of 3,000 conversational pairs to teach the model a specific conversational style and knowledge base. This process, known as fine-tuning, is a cornerstone of modern NLP, allowing us to adapt massive pre-trained models for specific tasks without the prohibitive cost of training them from scratch. By the end of this notebook, you will have a functional chatbot customized with our data and a clear understanding of the end-to-end LLM fine-tuning workflow.
    </div>

<div style="font-size: 20px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        The Core Objective
    </div>
    <div style="font-size: 16px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        Our primary objective is to demonstrate the power of transfer learning in NLP. We will take the `distilgpt2` model—a smaller, faster variant of GPT-2 that is perfect for experimentation in environments like Kaggle—and train it further on a custom question-and-answer dataset. We will then compare the responses of the original, base model with our newly fine-tuned model to concretely measure the improvement and see how the model's personality and knowledge have shifted.
    </div>

 <div style="font-size: 20px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Methodology at a Glance
    </div>
    <div style="font-size: 16px; line-height: 1.7; text-align: justify;">
        Our approach is structured into a clear, step-by-step process:
        <div style="margin-top: 15px; padding-left: 20px;">
            <div style="margin-bottom: 10px;"><b>1. Environment Setup:</b> We will import essential libraries like PyTorch and Hugging Face's `transformers` and `datasets`.</div>
            <div style="margin-bottom: 10px;"><b>2. Data Loading & Preparation:</b> We will load the 3k conversations dataset and format it into a single text sequence per conversation, making it suitable for training a causal language model like GPT-2.</div>
            <div style="margin-bottom: 10px;"><b>3. Model & Tokenizer Initialization:</b> We will load the pre-trained `distilgpt2` model and its corresponding tokenizer, setting up the necessary configurations for training.</div>
            <div style="margin-bottom: 10px;"><b>4. Fine-Tuning:</b> Using the Hugging Face `Trainer` API, we will fine-tune the model on our prepared dataset. This is where the model learns the new conversational patterns.</div>
            <div style="margin-bottom: 10px;"><b>5. Inference & Evaluation:</b> Finally, we'll test our new chatbot! We will provide it with prompts and compare its generated answers against those from the original model to see our improvements in action.</div>
        </div>
    </div>

</div>


<div style="background-color: #013220; color: #EAEAEA; border-radius: 15px; padding: 30px; font-family: 'Helvetica Neue', sans-serif; border: 1px solid #388E3C;">

 <div style="text-align: center; font-size: 28px; font-weight: bold; color: #A5D6A7; padding-bottom: 15px; border-bottom: 2px solid #388E3C; margin-bottom: 25px;">
        Step 2: Environment Setup & Loading the Base Model/Tokenizer
    </div>

 <div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Objective of This Step
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        In this step, we prepare our Kaggle environment for working with Large Language Models using the Hugging Face <code>transformers</code> library alongside <code>PyTorch</code>. This includes installing any required dependencies, importing essential packages, and initializing the building blocks of our project — the pre-trained base model and its tokenizer.
        The tokenizer is responsible for converting human-readable text into a sequence of tokens (numerical IDs) that the model can understand, while the model itself is the neural network that processes those tokens to generate text predictions.
    </div>

<div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Why We Use DistilGPT-2
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        We will use <code>distilgpt2</code>, which is a distilled (smaller and faster) version of GPT-2. This choice allows us to train and experiment quickly in Kaggle's GPU environment without exhausting memory or exceeding time limits, while still benefiting from the rich language understanding learned during GPT-2's original training.
    </div>

<div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Actions We Will Perform
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify;">
        <div style="padding-left: 20px;">
            <div style="margin-bottom: 10px;"><b>1.</b> Import the necessary Python packages: <code>transformers</code>, <code>torch</code>, and optionally <code>datasets</code>.</div>
            <div style="margin-bottom: 10px;"><b>2.</b> Initialize the model <code>distilgpt2</code> using the Hugging Face model hub.</div>
            <div style="margin-bottom: 10px;"><b>3.</b> Load the corresponding tokenizer so we can transform raw text into token IDs for the model to process.</div>
            <div style="margin-bottom: 10px;"><b>4.</b> Verify that the model and tokenizer are ready for fine-tuning on our custom dataset in the next steps.</div>
        </div>
    </div>

</div>


In [2]:


# Install the Hugging Face Transformers library (if not already available in Kaggle)
#!pip install transformers datasets --quiet

# Import required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer for distilgpt2
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Load the base model (DistilGPT-2) for Causal Language Modeling
model = AutoModelForCausalLM.from_pretrained("distilgpt2").to(device)

# Verify model and tokenizer are ready
sample_text = "Hello, how are you?"
encoded_input = tokenizer.encode(sample_text, return_tensors="pt").to(device)
print("Sample token IDs:", encoded_input)


Using device: cpu


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-10-02 08:25:29.498554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759393529.737407      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759393529.807197      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Sample token IDs: tensor([[15496,    11,   703,   389,   345,    30]])


In [3]:
import pandas as pd

# Dataset path in Kaggle
dataset_path = "/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv"

# Load CSV file into a Pandas DataFrame
df = pd.read_csv(dataset_path)

# Show basic info about the dataset
print("Dataset Shape:", df.shape)
print("\nColumn Names:", df.columns.tolist())

# Display first few rows
print("\nSample Rows:")
print(df.head())

# Check for missing values
print("\nMissing Values Per Column:")
print(df.isnull().sum())

# Show basic statistics (if numerical columns exist)
print("\nDataset Statistics:")
print(df.describe(include='all'))


Dataset Shape: (3725, 3)

Column Names: ['Unnamed: 0', 'question', 'answer']

Sample Rows:
   Unnamed: 0                             question  \
0           0               hi, how are you doing?   
1           1        i'm fine. how about yourself?   
2           2  i'm pretty good. thanks for asking.   
3           3    no problem. so how have you been?   
4           4     i've been great. what about you?   

                                     answer  
0             i'm fine. how about yourself?  
1       i'm pretty good. thanks for asking.  
2         no problem. so how have you been?  
3          i've been great. what about you?  
4  i've been good. i'm in school right now.  

Missing Values Per Column:
Unnamed: 0    0
question      0
answer        0
dtype: int64

Dataset Statistics:
         Unnamed: 0           question             answer
count   3725.000000               3725               3725
unique          NaN               3510               3512
top             NaN  wha

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


<div style="background-color: #013220; color: #EAEAEA; border-radius: 15px; padding: 30px; font-family: 'Helvetica Neue', sans-serif; border: 1px solid #388E3C;">

<div style="text-align: center; font-size: 28px; font-weight: bold; color: #A5D6A7; padding-bottom: 15px; border-bottom: 2px solid #388E3C; margin-bottom: 25px;">
        Step 3: Data Preparation for Fine-Tuning
    </div>

<div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Objective of This Step
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        The core objective here is to transform our structured dataset (with 'question' and 'answer' columns) into a format suitable for training a causal language model. GPT-2 learns by predicting the next token in a continuous sequence of text. Our current two-column format isn't a continuous sequence, so we must combine them into one. We will create a single text string for each row that clearly represents the conversational flow.
    </div>

<div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Our Formatting Strategy
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify; margin-bottom: 20px;">
        To help the model learn the structure of a conversation, we will format each row into a single string like this:
        <br><br>
        <code>"&lt;s&gt;[Q]: How are you? [A]: I'm doing great, thanks!&lt;/s&gt;"</code>
        <br><br>
        Here, <code>&lt;s&gt;</code> and <code>&lt;/s&gt;</code> represent the <b>start</b> and <b>end</b> of a single conversational exchange. This special token (known as the EOS or End-of-Sequence token) is crucial because it teaches the model when a thought or response is complete, which is vital for generating coherent answers during inference.
    </div>

<div style="font-size: 18px; font-weight: bold; color: #81C784; margin-top: 25px; margin-bottom: 10px;">
        Actions We Will Perform
    </div>
    <div style="font-size: 15px; line-height: 1.7; text-align: justify;">
        <div style="padding-left: 20px;">
            <div style="margin-bottom: 10px;"><b>1. Combine Columns:</b> We will merge the 'question' and 'answer' columns into a new column, applying our formatting rule.</div>
            <div style="margin-bottom: 10px;"><b>2. Configure Tokenizer:</b> Since <code>distilgpt2</code> does not have a default padding token, we will set its padding token to be the same as its end-of-sequence (EOS) token. This is a standard practice for this model.</div>
            <div style="margin-bottom: 10px;"><b>3. Create a Dataset Object:</b> We will convert our Pandas DataFrame into a Hugging Face <code>Dataset</code> object for efficient processing and tokenization.</div>
        </div>
    </div>

</div>


In [4]:
# Step 3: Data Preparation for Fine-Tuning



from datasets import Dataset

# --- 1. Format the data into a single string per row ---
# We will format our data as "<s>[Q]: {question} [A]: {answer}</s>"
# <s> and </s> are special tokens that mark the start and end of a sequence.
# This helps the model understand the structure of a complete conversational turn.

def format_conversation(row):
    return f"<s>[Q]: {row['question']} [A]: {row['answer']}</s>"

# Apply the formatting to the DataFrame
df['text'] = df.apply(format_conversation, axis=1)

print("Sample of formatted data:")
print(df['text'].head())


# --- 2. Configure the tokenizer ---
# The distilgpt2 model doesn't have a default padding token.
# We'll use the end-of-sequence (EOS) token as our padding token.
# This is a common practice for GPT-style models.
tokenizer.pad_token = tokenizer.eos_token


# --- 3. Create a Hugging Face Dataset object ---
# The Hugging Face Trainer API works best with 'Dataset' objects.
# We will convert our formatted text column from the pandas DataFrame
# into this specialized format.

# First, create a new DataFrame with only the text we need
training_data = pd.DataFrame({'text': df['text']})

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(training_data)

# --- 4. Tokenize the entire dataset ---
# We'll apply the tokenizer to all our text examples. The tokenizer will
# convert the text into numerical IDs that the model can understand.
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

print("\nDataset has been processed and tokenized.")
print("A single example from the tokenized dataset:")
print(tokenized_dataset[0])


Sample of formatted data:
0    <s>[Q]: hi, how are you doing? [A]: i'm fine. ...
1    <s>[Q]: i'm fine. how about yourself? [A]: i'm...
2    <s>[Q]: i'm pretty good. thanks for asking. [A...
3    <s>[Q]: no problem. so how have you been? [A]:...
4    <s>[Q]: i've been great. what about you? [A]: ...
Name: text, dtype: object


Map:   0%|          | 0/3725 [00:00<?, ? examples/s]


Dataset has been processed and tokenized.
A single example from the tokenized dataset:
{'input_ids': [27, 82, 36937, 48, 5974, 23105, 11, 703, 389, 345, 1804, 30, 685, 32, 5974, 1312, 1101, 3734, 13, 703, 546, 3511, 30, 3556, 82, 29, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1